In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sridhar\\Desktop\\DATA ANALYTICS PROJECTS\\DATA SCIENCE\\End-To-End-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sridhar\\Desktop\\DATA ANALYTICS PROJECTS\\DATA SCIENCE\\End-To-End-ML-Project'

In [5]:
# Purpose: Definition of the DataTransformationConfig data class.

from dataclasses import dataclass  # Importing the dataclass decorator.
from pathlib import Path  # Object-oriented interface to filesystem paths.

@dataclass(frozen=True)
class DataTransformationConfig:
    """
    Data class for configuration related to data transformation.

    Attributes:
        root_dir (Path): Root directory for data transformation artifacts.
        date_path (Path): Path to the input data file for transformation.
    """
    root_dir: Path
    data_path: Path

In [6]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories

In [7]:
# File: configuration_manager.py
# Purpose: Definition of the ConfigurationManager class for managing project configurations.

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):
        """
        Initializes the ConfigurationManager with default or provided file paths.

        Args:
            config_filepath (Path, optional): Path to the configuration file. Defaults to CONFIG_FILE_PATH.
            params_filepath (Path, optional): Path to the parameters file. Defaults to PARAMS_FILE_PATH.
            schema_filepath (Path, optional): Path to the schema file. Defaults to SCHEMA_FILE_PATH.
        """
        # Reading configuration, parameters, and schema files using read_yaml function.
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Creating the root directory for project artifacts.
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        """
        Retrieves the configuration for data transformation.

        Returns:
            DataTransformationConfig: Data class containing data transformation configuration.
        """
        # Extracting data transformation configuration from the overall project configuration.
        config = self.config.data_transformation

        # Creating the root directory for data transformation artifacts.
        create_directories([config.root_dir])

        # Creating a DataTransformationConfig object with the extracted configuration.
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [8]:
import os
from mlproject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [9]:
# Purpose: Definition of the DataTransformation class for handling data transformation.

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        """
        Initializes the DataTransformation object with the provided configuration.

        Args:
            config (DataTransformationConfig): DataTransformationConfig object containing data transformation settings.
        """
        self.config = config

    ## Note: You can add different data transformation techniques such as Scaler, PCA, and all
    # You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_splitting because this data is already cleaned up

    def train_test_splitting(self):
        """
        Performs train-test splitting on the data and saves the resulting sets to CSV files.
        """
        # Reading the data file into a pandas DataFrame.
        data = pd.read_csv(self.config.data_path)

        # Splitting the data into training and test sets using a default split ratio of (0.75, 0.25).
        train, test = train_test_split(data)

        # Saving the training and test sets to CSV files in the root directory for data transformation artifacts.
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        # Logging information about the split.
        logger.info("Split data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        # Printing the shapes of the resulting sets.
        print(train.shape)
        print(test.shape)


In [10]:
# Purpose: Main script or module for the machine learning project.

try:
    # Creating ConfigurationManager to manage project configurations.
    config = ConfigurationManager()

    # Retrieving data transformation configuration.
    data_transformation_config = config.get_data_transformation_config()

    # Creating DataTransformation object with the obtained configuration.
    data_transformation = DataTransformation(config=data_transformation_config)

    # Performing data transformation: train-test splitting.
    data_transformation.train_test_splitting()

except Exception as e:
    # Handling and re-raising any exceptions that occur during the process.
    raise e


[2023-12-02 21:33:45,932: INFO: common: YAML file: config\config.yaml loaded successfully]
[2023-12-02 21:33:45,939: INFO: common: YAML file: params.yaml loaded successfully]
[2023-12-02 21:33:45,956: INFO: common: YAML file: schema.yaml loaded successfully]
[2023-12-02 21:33:45,958: INFO: common: Created directory at: artifacts]
[2023-12-02 21:33:45,961: INFO: common: Created directory at: artifacts/data_transformation]
[2023-12-02 21:33:46,032: INFO: 1809509712: Split data into training and test sets]
[2023-12-02 21:33:46,036: INFO: 1809509712: (1199, 12)]
[2023-12-02 21:33:46,039: INFO: 1809509712: (400, 12)]
(1199, 12)
(400, 12)
